# MiniMind 模型

作为轻量级语言模型框架的典范，MiniMind项目通过其开源文档展现了"轻架构而全功能"的设计哲学。与多数基于传统Transformer Decoder架构的改良方案不同，该项目创新性地整合了多项前沿技术：包括提升位置编码效果的RoPE旋转位置编码机制、优化计算资源的共享MoE（混合专家）架构等创新设计。值得关注的是，相较于其他依赖多平台并行计算库（如LLaMA、DeepSpeed、Megatron-LM等）的开源LLM，MiniMind坚持使用纯PyTorch实现，这种技术选型显著降低了框架理解门槛，使其成为研究语言模型底层机制的优质教学范本。

**不过需要指出的是，当前版本存在代码可读性方面的提升空间。部分核心模块的注释缺失，以及某些刻意追求代码简写（如过度使用计算优化算法等）的编程风格，与项目宣称的"让每个学习者都能透彻理解每行代码"的初衷存在一定偏差。尽管如此，MiniMind仍不失为掌握现代语言模型实现细节的优秀学习对象。下文将从架构设计到模块实现，逐层解析该项目的技术细节。**

## MiniMind 官方架构图：

![MiniMind Struct](img/MiniMind-Struct.png)

---

## MiniMind 配置

---

In [ ]:
from transformers import PretrainedConfig
from typing import List

class LMConfig(PretrainedConfig):
    model_type = "minimind"

    def __init__(
            self,
            dim: int = 512,
            n_layers: int = 8,
            n_heads: int = 8,
            n_kv_heads: int = 2,
            vocab_size: int = 6400,
            hidden_dim: int = None,
            multiple_of: int = 64,
            norm_eps: float = 1e-5,
            max_seq_len: int = 8192,
            rope_theta: int = 1e6,
            dropout: float = 0.0,
            flash_attn: bool = True,
            ####################################################
            # 配置 MoE 的相关信息
            ####################################################
            use_moe: bool = False,
            ####################################################
            num_experts_per_tok: int = 2,
            n_routed_experts: int = 4,
            n_shared_experts: bool = True,
            scoring_func: str = 'softmax',
            aux_loss_alpha: float = 0.1,
            seq_aux: bool = True,
            norm_topk_prob: bool = True,
            **kwargs,
    ):
        self.dim = dim                                 # 词嵌入维度
        self.n_layers = n_layers                       # 解码器层数
        self.n_heads = n_heads                         # GQA 注意力的头数，要保证其为 n_kv_heads 的整数倍
        self.n_kv_heads = n_kv_heads                   # GQA 对应的 kv 头数
        self.vocab_size = vocab_size                   # 词表大小
        self.hidden_dim = hidden_dim                   # 隐藏维度
        self.multiple_of = multiple_of                 # SwiGLU 使用的 multiple_of 倍数参数，这里取的值为 2 的 n 次方
        self.norm_eps = norm_eps                       # 一个很小的数，用于规范化层，防止除零
        self.max_seq_len = max_seq_len                 # 序列的最大长度
        self.rope_theta = rope_theta                   # RoPE 旋转位置编码的超参数
        self.dropout = dropout                         # Dropout 比率，防止过拟合，但一般用不到
        self.flash_attn = flash_attn                   # 是否使用 pytorch 的 flash attention 实现
        ####################################################
        # Here are the specific configurations of MOE
        # When use_moe is false, the following is invalid
        ####################################################
        self.use_moe = use_moe                         # 是否使用 MoE
        self.num_experts_per_tok = num_experts_per_tok # 每个token选择的专家数量，就是 top_k 的大小
        self.n_routed_experts = n_routed_experts       # 总的专家数量
        self.n_shared_experts = n_shared_experts       # 共享专家
        self.scoring_func = scoring_func               # 评分函数，默认为 'softmax'
        self.aux_loss_alpha = aux_loss_alpha           # 辅助损失的 alpha 参数
        self.seq_aux = seq_aux                         # 是否在序列级别上计算辅助损失
        self.norm_topk_prob = norm_topk_prob           # 是否标准化 top-k 概率
        super().__init__(**kwargs)

## MiniMind 代码解析

---

### 1. 引入必要的库

In [ ]:
import math
import struct
import inspect
import time

from typing import Any, Optional, Tuple, List
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from transformers import PreTrainedModel
from transformers.modeling_outputs import CausalLMOutputWithPast

### 2. 定义 RMSNorm

这是一个典型的 RMSNorm 模块，公式如下：

$$
RMSNorm(x) = \gamma \odot \frac{x}{\sqrt{\frac{1}{d}\sum^{d}_{i = 0}{x_i^{2}}+\epsilon}}
$$

其中：

- $x$ 是输入向量，维度为 $d$。
- $\epsilon$ 是一个很小的数，用于防止除零。
- $\gamma$ 是可学习的缩放参数，维度为 $d$。
- $\odot$ 表示元素级的乘法操作。

In [ ]:
class RMSNorm(torch.nn.Module):
    def __init__(self, dim: int, eps: float):
        """
        RMS Norm

        :param dim: 嵌入维度大小
        :param eps: 一个很小的值，防止除零
        """
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        return self.weight * (x.float() * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)).type_as(x)

### 3. 定义位置编码相关的函数

MiniMind 使用的位置编码为**旋转位置编码（Rotary Position Embedding, RoPE）**，下面是相关公式：

**（1）频率基底计算**

定义频率基底向量$\theta_{j}$：

$$
\theta_{j} = \frac{1}{\gamma^{\frac{2j}{d}}}，其中 j=0,1,...,\frac{d}{2}-1
$$

- $d$：向量的维度
- $\gamma$：超参数，控制频率衰减速度

**（2）位置旋转矩阵**

对于位置$m$，定义旋转矩阵作用于向量的第 $2j$ 和 $2j+1$ 维度：

$$
R_{m}^{(j)} =
\begin{bmatrix}
\cos(m\theta_{j}) & -\sin(m\theta_{j}) \\
\sin(m\theta_{j}) & \cos(m\theta_{j})
\end{bmatrix}
$$

或用复数形式等价表示为：

$$
R_{m}^{(j)} = e^{im\theta_{j}}
$$

其中 $i$ 是虚数单位。

**（3）应用到查询/键向量**

给定查询向量 $q$ 或键向量 $k$（维度 $d$ ），将其拆分为 $\frac{d}{2}$ 个二维块：

$$
q = [q_{0}, q_{1}, q_{2},...,q_{d-2},q_{d-1}]
$$

对每个块应用旋转：

$$
RoPE(q,m) = [q_{0}\cos(m\theta_{0}) - q_{1}\sin(m\theta_{0}), q_{0}\sin(m\theta_{0}) - q_{1}\cos(m\theta_{0}), ...]
$$

**RoPE的优势**

- **相对位置编码**：无需显式学习位置关系，通过旋转矩阵的几何性质自然捕捉相对位置。
- **长序列友好**：旋转操作具有周期性（如 $\theta_{j}$ 的衰减设计），避免位置编码数值过大或过小。

In [ ]:
def precompute_pos_cis(dim: int, end: int = int(32 * 1024), theta: float = 1e6):
    """
    预计算RoPE（旋转位置编码）的复数形式位置编码矩阵

    :param dim: 嵌入维度（需要是偶数）
    :param end: 最大序列长度（默认32K tokens）
    :param theta: 频率计算的基值（默认为1e6，与Transformer-XL风格一致）
    :return: 复数形式的位置编码矩阵，形状为 (end, dim//2)
    """

    # 计算每个维度位置的频率因子（公式中的 θ_j）
    # 频率因子按公式 1/(theta^(2j/dim)) 计算，j为维度索引（仅计算前dim//2个）
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))

    # 生成位置序列 [0, 1, 2, ..., end-1]
    t = torch.arange(end, device=freqs.device)

    # 计算所有位置和维度的角度矩阵（外积）
    # 结果形状: (end, dim//2)
    freqs = torch.outer(t, freqs).float()

    # 将角度转换为复数形式（模为1，角度为freqs的复数）
    # 每个位置对应的旋转复数：cis(pos * theta_j) = cos(pos*theta_j) + i*sin(pos*theta_j)
    pos_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64

    return pos_cis


def apply_rotary_emb(xq, xk, pos_cis):
    """
    应用旋转位置编码到查询（Query）和键（Key）矩阵

    :param xq: 查询矩阵，形状为 (..., seq_len, num_heads, head_dim)
    :param xk: 键矩阵，形状与xq相同
    :param pos_cis: 预计算的位置编码矩阵
    :return: 旋转后的查询和键矩阵（保持原始形状）
    """

    def unite_shape(pos_cis, x):
        """调整位置编码矩阵形状以匹配输入张量的维度"""
        ndim = x.ndim
        # 确保位置编码在序列长度和特征维度上对齐
        assert pos_cis.shape == (x.shape[1], x.shape[-1])
        # 创建广播用形状，例如将 (seq_len, dim) 转换为 (1, seq_len, 1, ..., 1, dim)
        shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
        return pos_cis.view(*shape)

    # 将最后两个维度（实部、虚部）转换为复数形式
    # 原始形状: (..., seq_len, num_heads, head_dim)
    # 转换后: (..., seq_len, num_heads, head_dim//2)（复数形式）
    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))

    # 调整位置编码矩阵形状以便广播计算
    pos_cis = unite_shape(pos_cis, xq_)

    # 执行复数乘法实现旋转操作（相当于同时旋转Q和K）
    # 旋转后恢复为实数形式，形状: (..., seq_len, num_heads, head_dim)
    xq_out = torch.view_as_real(xq_ * pos_cis).flatten(3)
    xk_out = torch.view_as_real(xk_ * pos_cis).flatten(3)

    return xq_out.type_as(xq), xk_out.type_as(xk)

### 4. 定义 GQA （组查询注意力）

**GQA（Grouped-Query Attention）** 是Transformer中多头注意力机制的优化变体，它通过将多个查询头进行分组，并在每组内共享同一组键和值，从而在降低计算成本和内存占用的同时，较好地平衡了模型的性能和效率，尤其适用于长文本处理和大规模推理任务。

![GQA Struct](img/GQA-Struct.svg)

In [ ]:
def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    复制并扩充 KV 矩阵，等价于 `torch.repeat_interleave(x, dim=2, repeats=n_rep)`

    :param x: 传入的数据
    :param n_rep:  重复的数量
    :return: 复制完成的矩阵
    """
    bs, slen, n_kv_heads, head_dim = x.shape
    if n_rep == 1:
        return x
    return (
        x[:, :, :, None, :]
        .expand(bs, slen, n_kv_heads, n_rep, head_dim)
        .reshape(bs, slen, n_kv_heads * n_rep, head_dim)
    )


class Attention(nn.Module):
    def __init__(self, args: LMConfig):
        super().__init__()

        # 获取 kv 对应的头数
        self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads

        # 这里需要 kv 的头数能够将 q 的头数除尽，即需要 q 的头数是 kv 的整数倍
        assert args.n_heads % self.n_kv_heads == 0
        self.n_local_heads = args.n_heads
        self.n_local_kv_heads = self.n_kv_heads

        # 获取组数
        self.n_rep = self.n_local_heads // self.n_local_kv_heads

        # 根据嵌入维度，自动计算头维度：头维度 = 嵌入维度 / q头数
        self.head_dim = args.dim // args.n_heads

        # 定义 q,k,v 矩阵权重
        self.wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wk = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False)
        self.wv = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False)

        # 定义多有注意力的 w 权重
        self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False)

        # Dropout 防止过拟合
        self.attn_dropout = nn.Dropout(args.dropout)
        self.resid_dropout = nn.Dropout(args.dropout)
        self.dropout = args.dropout

        # 这里判断 pytorch 是否有 scaled_dot_product_attention 这个实现。
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention') and args.flash_attn
        # print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")

        # 构建上三角矩阵掩码，并注册为缓存
        mask = torch.full((1, 1, args.max_seq_len, args.max_seq_len), float("-inf"))
        mask = torch.triu(mask, diagonal=1)
        self.register_buffer("mask", mask, persistent=False)

    def forward(self,
                x: torch.Tensor,
                pos_cis: torch.Tensor,
                past_key_value: Optional[Tuple[torch.Tensor, torch.Tensor]] = None,
                use_cache=False):

        bsz, seq_len, _ = x.shape

        # 计算 qkv
        # q   的维度为 (bsz, seq_len, n_local_heads * head_dim)
        # k/v 的维度为 (bsz, seq_len, n_local_kv_heads * head_dim)
        xq, xk, xv = self.wq(x), self.wk(x), self.wv(x)

        # 更改维度
        # q   的维度为 (bsz, seq_len, n_local_heads    , head_dim)
        # k/v 的维度为 (bsz, seq_len, n_local_kv_heads , head_dim)
        xq = xq.view(bsz, seq_len, self.n_local_heads, self.head_dim)
        xk = xk.view(bsz, seq_len, self.n_local_kv_heads, self.head_dim)
        xv = xv.view(bsz, seq_len, self.n_local_kv_heads, self.head_dim)

        # 应用旋转位置嵌入到 q/v ，维度不变
        xq, xk = apply_rotary_emb(xq, xk, pos_cis)

        # kv_cache实现
        if past_key_value is not None:
            xk = torch.cat([past_key_value[0], xk], dim=1)
            xv = torch.cat([past_key_value[1], xv], dim=1)
        past_kv = (xk, xv) if use_cache else None

        # 此处两个目的
        # 1. 扩充 k/v 张量使其与 q 张量一致
        # 2. 转换 qkv 三个张量的维度为 (bsz, n_local_heads, seq_len, head_dim)
        xq, xk, xv = (
            xq.transpose(1, 2),
            repeat_kv(xk, self.n_rep).transpose(1, 2),
            repeat_kv(xv, self.n_rep).transpose(1, 2)
        )

        # 这里是判断是否使用 pytorch 自带的 flash attention 函数
        if self.flash and seq_len != 1:
            dropout_p = self.dropout if self.training else 0.0
            output = F.scaled_dot_product_attention(
                xq, xk, xv,
                attn_mask=None,
                dropout_p=dropout_p,
                is_causal=True
            )
        else:
            # 如果不使用自带的 flash attention ，就手动计算

            # 计算 qk^t/sqrt(head_dim) ，计算结果维度为 (bsz, n_local_heads, seq_len, seq_len)
            scores = (xq @ xk.transpose(-2, -1)) / math.sqrt(self.head_dim)

            # 对计算结果做遮掩
            scores += self.mask[:, :, :seq_len, :seq_len]

            # 对遮掩后的结果做 softmax
            scores = F.softmax(scores.float(), dim=-1).type_as(xq)

            # 防止过拟合
            scores = self.attn_dropout(scores)

            # 使用矩阵乘法求最终输出，维度为 (bsz, n_local_heads, seq_len, head_dim)
            output = scores @ xv

        # 将最终维度转换为 (bsz, seq_len, dim)
        output = output.transpose(1, 2).reshape(bsz, seq_len, -1)

        # 防止过拟合
        output = self.resid_dropout(self.wo(output))

        return output, past_kv

### 5. 前馈神经网络

前馈神经网络是一个两层的全连接神经网络，这里使用的前馈神经网络激活函数是 SwiGLU，其公式如下：

$$
SwiGLU(x,W,V) = x \cdot sigmoid (\beta x)\otimes(xV)
$$

此处令 $\beta = 1$ ， 那么 “$\otimes$” 号左侧部分便是 `SiLU` 激活函数。

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, config: LMConfig):
        super().__init__()

        # 这里是依据 multiple_of 计算隐藏层大小，通过确保隐藏层的大小是特定值的倍数，可以减少计算过程中的冗余操作，提高计算效率。
        # 同时，这也可能有助于减少内存碎片，提高内存的使用效率。
        if config.hidden_dim is None:
            hidden_dim = 4 * config.dim
            hidden_dim = int(2 * hidden_dim / 3)
            config.hidden_dim = config.multiple_of * ((hidden_dim + config.multiple_of - 1) // config.multiple_of)

        # 第一层权重
        self.w1 = nn.Linear(config.dim, config.hidden_dim, bias=False)
        # 第二层权重
        self.w2 = nn.Linear(config.hidden_dim, config.dim, bias=False)
        # 公式中对应的 V 矩阵
        self.w3 = nn.Linear(config.dim, config.hidden_dim, bias=False)

        # 防止过拟合
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        return self.dropout(self.w2(F.silu(self.w1(x)) * self.w3(x)))

### 6. MoE 门控（路由）

MoE 门控（路由）网络结构如下所示：

![MoE Router](img/MoE-Router.svg)

#### 负载均衡辅助损失函数

该门控网络中还定义了两种辅助损失函数：

**预定义：**

- $b$ 表示批次
- $n$ 表示专家数量
- $\alpha$ 表示权重系数

**（1） 序列级别辅助损失**

$$
L_{aux} = \alpha \cdot \frac{1}{b} \sum_{i=1}^{b} \left(\sum_{j=1}^{n} p_{i,j} \cdot ce_{i,j} \right)
$$

其中：
- $p_{i,j}$ 表示样本 $i$ 中专家 $j$ 的平均路由概率
- $ce_{i,j}$ 表示样本 $i$ 中专家 $j$ 的归一化选中次数

**（2） 全局级别辅助损失**

$$
L_{aux} = \alpha \cdot \left(\sum_{j=1}^{n} p_{j} \cdot (ce_{j} \cdot n) \right)
$$

其中：
- $p_{j}$ 表示专家 $j$ 的全局平均路由概率
- $ce_{j}$ 表示专家 $j$ 全局平均被选中的概率

上述两者辅助损失基于以下约束设计：

$$
L_{aux} \propto \sum_{i=1}^{N} P_{i} \cdot f_{i}
$$

- $p_{i}$ 表示平均路由概率，$f_{i}$ 表示频率缩放因子
- 均匀性目标：当专家被均匀选中时，$f_{i} = 1$，此时损失退化为 $\sum P_{i} = 1$，达到最小值
- 惩罚机制：若某些专家被过度使用（$f_{i} > 1$）或欠使用（$f_{i} < 1$），损失会通过 $P_{i} \cdot f_{i}$ 放大差异，迫使路由网络平衡负载。

In [ ]:
class MoEGate(nn.Module):
    def __init__(self, config: LMConfig):
        super().__init__()
        self.config = config
        self.top_k = config.num_experts_per_tok
        self.n_routed_experts = config.n_routed_experts

        self.scoring_func = config.scoring_func
        self.alpha = config.aux_loss_alpha
        self.seq_aux = config.seq_aux

        self.norm_topk_prob = config.norm_topk_prob
        self.gating_dim = config.dim

        # 这里使用了 kaiming_uniform 来初始化门控网络权重
        self.weight = nn.Parameter(torch.empty((self.n_routed_experts, self.gating_dim)))
        self.reset_parameters()

    def reset_parameters(self) -> None:
        import torch.nn.init as init
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))

    def forward(self, hidden_states):

        bsz, seq_len, h = hidden_states.shape

        # 将隐状态的维度转换为 (bsz * seq_len, h)
        hidden_states = hidden_states.view(-1, h)

        # 经过门控网络的 FNN 输出 logits， 维度为 (bsz * seq_len, n_routed_experts)
        logits = F.linear(hidden_states, self.weight, None)
        # 这里就是使用 softmax 将 logits 转换为概率分布，维度保持不变
        if self.scoring_func == 'softmax':
            scores = logits.softmax(dim=-1)
        else:
            raise NotImplementedError(f'insupportable scoring function for MoE gating: {self.scoring_func}')

        # 取 top_k 的专家
        # topk_weight： top_k 专家的权重，维度为 (bsz * seq_len, top_k)
        # topk_idx   ： top_k 专家的索引，维度为 (bsz * seq_len, top_k)
        topk_weight, topk_idx = torch.topk(scores, k=self.top_k, dim=-1, sorted=False)

        # 这里是判断是否将 topk_weight 归一化
        if self.top_k > 1 and self.norm_topk_prob:
            denominator = topk_weight.sum(dim=-1, keepdim=True) + 1e-20
            topk_weight = topk_weight / denominator

        # 下面的过程是求辅助损失的
        if self.training and self.alpha > 0.0:

            # 数据准备
            scores_for_aux = scores
            aux_topk = self.top_k
            topk_idx_for_aux_loss = topk_idx.view(bsz, -1)

            if self.seq_aux:
                # 计算序列级别辅助损失

                scores_for_seq_aux = scores_for_aux.view(bsz, seq_len, -1)
                ce = torch.zeros(bsz, self.n_routed_experts, device=hidden_states.device)

                # 统计每个样本中每个专家被选中的次数，并归一化到期望均匀分布的比例（除以 seq_len * top_k / n_experts）
                ce.scatter_add_(1, topk_idx_for_aux_loss,
                                torch.ones(bsz, seq_len * aux_topk, device=hidden_states.device)).div_(
                    seq_len * aux_topk / self.n_routed_experts)

                aux_loss = (ce * scores_for_seq_aux.mean(dim=1)).sum(dim=1).mean() * self.alpha
            else:
                # 计算全局级别辅助损失

                # 这里就是求每个专家平均被选中的概率
                mask_ce = F.one_hot(topk_idx_for_aux_loss.view(-1), num_classes=self.n_routed_experts)
                ce = mask_ce.float().mean(0)

                # 依公式计算
                Pi = scores_for_aux.mean(0)
                fi = ce * self.n_routed_experts
                aux_loss = (Pi * fi).sum() * self.alpha

        else:
            aux_loss = 0
        return topk_idx, topk_weight, aux_loss

### 7. MoE

MiniMind 使用的 MoE 与 Deepseek MoE 类似，都采用“共享专家” + “普通专家” 的模式，原理图如下：

![Minimind MoE Struct](img/MiniMinid-MoE-Struct.svg)

作者在 MoE 的实现中，使用了一些技巧来取出隐状态对应的专家并与其分配的权重进行相乘，有关这部分的内容会在代码注释中进行解释。**这里要特别指出原文中 `moe_infer()` 这个函数原代码作者基本上就是在炫技般的解决问题，在关键部分基本没有注释并隐藏自己的意图。** 参考优秀项目每个函数都会有极为详细的注释，这里需指出这就是一坨屎。（不是指代码是屎，而是指炫技而不留注释的行为是屎）

In [ ]:
class MOEFeedForward(nn.Module):
    def __init__(self, config: LMConfig):
        super().__init__()
        self.config = config

        # 定义专家，这里的专家就是直接使用更小的前馈神经网络组成
        self.experts = nn.ModuleList([
            FeedForward(config)
            for _ in range(config.n_routed_experts)
        ])

        # 定义路由
        self.gate = MoEGate(config)

        # 定义共享专家，这里只有一个共享专家
        if config.n_shared_experts is not None:
            self.shared_experts = FeedForward(config)

    def forward(self, x):

        identity = x
        orig_shape = x.shape
        bsz, seq_len, _ = x.shape

        # 使用门控机制选择专家
        # topk_idx 和 topk_weight 的维度为 (bsz * seq_len, top_k)
        # aux_loss 是一个常数
        topk_idx, topk_weight, aux_loss = self.gate(x)

        # 将 x 维度转换为 (bsz * seq_len, dim)
        # 方便后续数据处理，同时可以提高计算效率
        x = x.view(-1, x.shape[-1])

        # 将 top_k 的专家索引打平为一维，其维度为 (bsz * seq_len * top_k)
        flat_topk_idx = topk_idx.view(-1)

        # 训练模式和推理模式两种模式在数学上等价，二者都是加权求和，但实现方式不同：
        # 训练模式：牺牲内存效率，保证梯度信息的稳定性。
        # 推理模式：优化内存和计算效率，利用排序和批量处理加速。
        if self.training:

            # 训练模式下，重复输入数据
            # 复制后的 x 维度为 (bsz * seq_len * top_k, dim)
            # 其第一个维度与 flat_topk_idx 是一致的，为后续专家输出处理做准备
            x = x.repeat_interleave(self.config.num_experts_per_tok, dim=0)

            # 计算 top_k 专家的输出，这里由于输入的 x 是复制的，所以对于同一隐状态传给不同专家处理的值其实是相同的
            y = torch.empty_like(x, dtype=torch.float16)
            for i, expert in enumerate(self.experts):
                # 这里 flat_topk_idx == i 相当于筛选出有哪些隐状态在 top_k 的位置上的使用了当前第 i 个专家
                y[flat_topk_idx == i] = expert(x[flat_topk_idx == i]).to(y.dtype)  # 确保类型一致

            # 令专家的输出乘上对应的权重，然后求和。
            # (1) y.view(*topk_weight.shape, -1) 维度为 (bsz * seq_len, top_k, dim)
            # (2) topk_weight.unsqueeze(-1) 维度为 (bsz * seq_len, top_k, 1)
            # (1) * (2) 的维度为 (bsz * seq_len, top_k, dim)
            # 在维度 1 求和后，最终输出维度为 (bsz * seq_len, dim)
            y = (y.view(*topk_weight.shape, -1) * topk_weight.unsqueeze(-1)).sum(dim=1)

            # 将输出恢复为原始的形状，即 (bsz, seq_len, dim)
            y = y.view(*orig_shape)
        else:
            # 推理模式下，只选择最优专家
            y = self.moe_infer(x, flat_topk_idx, topk_weight.view(-1, 1)).view(*orig_shape)

        # 如果有共享专家，那么直接加上共享专家的值
        if self.config.n_shared_experts is not None:
            y = y + self.shared_experts(identity)

        # 辅助损失
        self.aux_loss = aux_loss

        return y

    @torch.no_grad() # 禁用梯度计算，减少内存占用并加速推理。
    def moe_infer(self, x, flat_expert_indices, flat_expert_weights):

        # 这是一个用累加操作的张量，作用类似于 sum
        expert_cache = torch.zeros_like(x)

        # flat_expert_indices 是一个一维数组，其维度为 (bsz * seq_len * top_k)
        # argsort() 的作用是返回张量沿指定维度排序后的【元素索引】，如：
        # [3,1,2] 排序后是 [1,2,3]，那么 argsort() 返回 [1,2,0]
        idxs = flat_expert_indices.argsort()

        # 该过程可以理解为给专家分配 token 序列中的某一段进行处理，比如：
        # 有 3 个专家：[0,1,2] ，flat_expert_indices = [1,0, 2,1, 1,2]
        # bincount() 用于统计输入张量中每个专家的出现次数，即返回 [1,3,2] （0 出现2次，1出现3次，2出现2次）
        # cumsum(0) 用于表示沿第一个维度（行）进行累积求和，即返回 [1,1+3,1+3+2] = [1,4,6]
        # 这就相当于：
        # 位于 [0,1) 的 token 分配给专家 0 处理
        # 位于 [1,3) 的 token 分配给专家 1 处理
        # 位于 [3,6) 的 token 分配给专家 2 处理
        tokens_per_expert = flat_expert_indices.bincount().cpu().numpy().cumsum(0)

        # 这里令 idxs 使用整数除法除以 top_k ，其实就是以 batch * seq_len 为单位分割 token
        # 即获取的是按顺序使用专家的 token 索引
        token_idxs = idxs // self.config.num_experts_per_tok

        # 例如当tokens_per_expert=[6, 15, 20, 26, 33, 38, 46, 52]
        # 当token_idxs=[3, 7, 19, 21, 24, 25,  4,  5,  6, 10, 11, 12...]
        # 意味着当token_idxs[:6] -> [3,  7, 19, 21, 24, 25,  4]位置的token都由专家0处理，token_idxs[6:15]位置的token都由专家1处理......
        for i, end_idx in enumerate(tokens_per_expert):

            # 取出开始位置的索引
            start_idx = 0 if i == 0 else tokens_per_expert[i - 1]
            if start_idx == end_idx:
                continue

            # 获取对应的专家
            expert = self.experts[i]

            # 获取该专家处理的 token 序列段
            exp_token_idx = token_idxs[start_idx:end_idx]
            expert_tokens = x[exp_token_idx]

            # 专家输出
            expert_out = expert(expert_tokens).to(expert_cache.dtype)

            # 零专家输出乘上对应的权重
            expert_out.mul_(flat_expert_weights[idxs[start_idx:end_idx]])

            # 使用 scatter_add_ 进行 sum 操作
            expert_cache.scatter_add_(0, exp_token_idx.view(-1, 1).repeat(1, x.shape[-1]), expert_out)

        return expert_cache

### 8. MiniMind（解码器）块

尽管作者将该模块命名为了 `MiniMindBlock` ，但其本质上就是一个 Decoder， 在其他开源模型中，也会将此类的块命名为 `TransformerBlock`。其结构如下图所示：

![MiniMind Block](img/MiniMind-Block.svg)


In [ ]:
class MiniMindBlock(nn.Module):
    def __init__(self, layer_id: int, config: LMConfig):
        super().__init__()

        # 这里主要是定义注意力相关的部分
        self.n_heads = config.n_heads
        self.dim = config.dim
        self.head_dim = config.dim // config.n_heads
        self.attention = Attention(config)

        # 这里定义的是归一化层相关的部分
        self.layer_id = layer_id
        self.attention_norm = RMSNorm(config.dim, eps=config.norm_eps)
        self.ffn_norm = RMSNorm(config.dim, eps=config.norm_eps)

        # 如果使用 MoE ，那么就将前馈神经网络替换 MoE 模块，否则就使用两层的前馈神经网络
        self.feed_forward = FeedForward(config) if not config.use_moe else MOEFeedForward(config)

    def forward(self, x, pos_cis, past_key_value=None, use_cache=False):

        # 获取 注意力返回结果 和 kv cache
        h_attn, past_kv = self.attention(
            self.attention_norm(x), # 这里进行了一次归一化
            pos_cis,
            past_key_value=past_key_value,
            use_cache=use_cache
        )

        # 残差网络相加
        h = x + h_attn

        # 归一化后再传入 前馈神经网络/MoE ，输出最终结果
        out = h + self.feed_forward(self.ffn_norm(h))
        return out, past_kv

### 9. MiniMind 模型实现

本文关注的是模型结构，`generate()` 和 `_stream()` 函数中的内容将不再关心和注释。

In [ ]:
class MiniMindLM(PreTrainedModel):
    config_class = LMConfig

    def __init__(self, params: LMConfig = None):
        self.params = params or LMConfig()
        super().__init__(self.params)
        self.vocab_size, self.n_layers = params.vocab_size, params.n_layers
        self.tok_embeddings = nn.Embedding(params.vocab_size, params.dim)
        self.dropout = nn.Dropout(params.dropout)
        self.layers = nn.ModuleList([MiniMindBlock(l, params) for l in range(self.n_layers)])
        self.norm = RMSNorm(params.dim, eps=params.norm_eps)
        self.output = nn.Linear(params.dim, params.vocab_size, bias=False)
        self.tok_embeddings.weight = self.output.weight

        # 预计算位置编码
        self.register_buffer("pos_cis",
                             precompute_pos_cis(dim=params.dim // params.n_heads, theta=params.rope_theta),
                             persistent=False)

        self.OUT = CausalLMOutputWithPast()

    def forward(self,
                input_ids: Optional[torch.Tensor] = None,
                past_key_values: Optional[List[Tuple[torch.Tensor, torch.Tensor]]] = None,
                use_cache: bool = False,
                **args):

        past_key_values = past_key_values or [None] * len(self.layers)
        start_pos = args.get('start_pos', 0)

        # 将输入嵌入词向量维度
        h = self.dropout(self.tok_embeddings(input_ids))

        # 获取位置编码基地，位置编码会在 MiniMindBlock 的注意力块中进行嵌入
        pos_cis = self.pos_cis[start_pos:start_pos + input_ids.size(1)]

        # 在 MiniMindBlock 块中逐层传递输出
        past_kvs = []
        for l, layer in enumerate(self.layers):
            h, past_kv = layer(
                h, pos_cis,
                past_key_value=past_key_values[l],
                use_cache=use_cache
            )
            past_kvs.append(past_kv)

        # 获取最终的 logits
        logits = self.output(self.norm(h))

        # 叠加所有层的辅助损失
        aux_loss = sum(l.feed_forward.aux_loss for l in self.layers if isinstance(l.feed_forward, MOEFeedForward))

        self.OUT.__setitem__('logits', logits)
        self.OUT.__setitem__('aux_loss', aux_loss)
        self.OUT.__setitem__('past_key_values', past_kvs)
        return self.OUT

    @torch.inference_mode()
    def generate(self, input_ids, eos_token_id=2, max_new_tokens=1024, temperature=0.75, top_p=0.90,
                 stream=False, rp=1., use_cache=True, pad_token_id=0, **args):
        # 流式生成
        if stream:
            return self._stream(input_ids, eos_token_id, max_new_tokens, temperature, top_p, rp, use_cache, **args)

        # 直接生成
        generated = []
        for i in range(input_ids.size(0)):
            non_pad = input_ids[i][input_ids[i] != pad_token_id].unsqueeze(0)
            out = self._stream(non_pad, eos_token_id, max_new_tokens, temperature, top_p, rp, use_cache, **args)
            tokens_list = [tokens[:, -1:] for tokens in out]
            gen = torch.cat(tokens_list, dim=-1) if tokens_list else non_pad
            full_sequence = torch.cat([non_pad, gen], dim=-1)
            generated.append(full_sequence)
        max_length = max(seq.size(1) for seq in generated)
        generated = [
            torch.cat(
                [seq, torch.full((1, max_length - seq.size(1)), pad_token_id, dtype=seq.dtype, device=seq.device)],
                dim=-1)
            for seq in generated
        ]
        return torch.cat(generated, dim=0)

    def _stream(self, input_ids, eos_token_id, max_new_tokens, temperature, top_p, rp, use_cache, **args):
        start, first_seq, past_kvs = input_ids.shape[1], True, None
        while input_ids.shape[1] < max_new_tokens - 1:
            if first_seq or not use_cache:
                out, first_seq = self(input_ids, past_key_values=past_kvs, use_cache=use_cache, **args), False
            else:
                out = self(input_ids[:, -1:], past_key_values=past_kvs, use_cache=use_cache,
                           start_pos=input_ids.shape[1] - 1, **args)
            logits, past_kvs = out.logits[:, -1, :], out.past_key_values
            logits[:, list(set(input_ids.tolist()[0]))] /= rp
            logits /= (temperature + 1e-9)
            if top_p is not None and top_p < 1.0:
                sorted_logits, sorted_indices = torch.sort(logits, descending=True, dim=-1)
                sorted_probs = F.softmax(sorted_logits, dim=-1)
                cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
                sorted_indices_to_remove[:, 0] = False
                indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
                logits[indices_to_remove] = -float('Inf')
            input_ids_next = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
            input_ids = torch.cat((input_ids, input_ids_next), dim=1)
            yield input_ids[:, start:]
            if input_ids_next.item() == eos_token_id:
                break